<a href="https://colab.research.google.com/github/kenyon-mitchell01/Payne_KCC2/blob/main/gninaworkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Setup

In [ ]:
!apt install openbabel

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
!conda install -q -y -c conda-forge openbabel

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')

In [ ]:
!pip install py3Dmol

In [ ]:
!wget https://downloads.sourceforge.net/project/smina/smina.static

In [ ]:
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina

In [ ]:
!chmod +x gnina

In [ ]:
!./gnina

# Example

In [ ]:
!wget http://files.rcsb.org/download/3ERK.pdb

In [ ]:
!grep ATOM 3ERK.pdb > rec.pdb
!obabel rec.pdb -Orec.pdb

In [ ]:
!grep SB4 3ERK.pdb > lig.pdb

In [ ]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':0.15}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})

In [ ]:
!obabel -:'C1CNCCC1n1cnc(c2ccc(cc2)F)c1c1ccnc(n1)N' -Ol2.sdf --gen2D

In [ ]:
v = py3Dmol.view()
v.addModel(open('l2.sdf').read())
v.setStyle({'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo()

In [ ]:
!obabel -:'C1CNCCC1n1cnc(c2ccc(cc2)F)c1c1ccnc(n1)N' -Ol3.sdf --gen3D

In [ ]:
v = py3Dmol.view()
v.addModel(open('l3.sdf').read())
v.setStyle({'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo()

# Simple Docking

In [ ]:
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb

In [ ]:
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0 -o docked.sdf.gz

In [ ]:
!gunzip docked.sdf.gz #older openbabel has trouble with gzipped files

In [ ]:
!obrms -firstonly lig.pdb docked.sdf

In [ ]:
import gzip
v = py3Dmol.view()
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docked.sdf','rt').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)

Run the necessary commands to dock `l3.sdf`, which is a generated conformer, instead of `lig.pdb` which is the crystal conformer.

In [ ]:
!./gnina

Evaluate RMSD of docked poses to crystal.

In [ ]:
!obrms

# Timing

In [ ]:
%%time
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0 --exhaustiveness 1 > /dev/null 2>&1

In [ ]:
%%time
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0 --exhaustiveness 4 > /dev/null 2>&1

In [ ]:
%%time
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0 --exhaustiveness 4 --cpu 1 > /dev/null 2>&1

In [ ]:
!cat /proc/cpuinfo

# Scoring

In [ ]:
!./gnina --score_only -r rec.pdb -l lig.pdb --verbosity=2

In [ ]:
!./gnina --help | grep scoring | head -3

In [ ]:
!./gnina --print_terms

In [ ]:
open('everything.txt','wt').write('''
1.0  ad4_solvation(d-sigma=3.6,_s/q=0.01097,_c=8)  desolvation, s/q is charge dependence
1.0  ad4_solvation(d-sigma=3.6,_s/q=0.0,_c=8)
1.0  electrostatic(i=1,_^=100,_c=8)	i is the exponent of the distance, see everything.h for details
1.0  electrostatic(i=2,_^=100,_c=8)
1.0  gauss(o=0,_w=0.5,_c=8)		o is offset, w is width of gaussian
1.0  gauss(o=3,_w=2,_c=8)
1.0  repulsion(o=0,_c=8)	o is offset of squared distance repulsion
1.0  hydrophobic(g=0.5,_b=1.5,_c=8)		g is a good distance, b the bad distance
1.0  non_hydrophobic(g=0.5,_b=1.5,_c=8)	value is linearly interpolated between g and b
1.0  vdw(i=4,_j=8,_s=0,_^=100,_c=8)	i and j are LJ exponents
1.0  vdw(i=6,_j=12,_s=1,_^=100,_c=8) s is the smoothing, ^ is the cap
1.0  non_dir_h_bond(g=-0.7,_b=0,_c=8)	good and bad
1.0  non_dir_anti_h_bond_quadratic(o=0.4,_c=8) like repulsion, but for hbond, don't use
1.0  non_dir_h_bond_lj(o=-0.7,_^=100,_c=8)	LJ 10-12 potential, capped at ^
1.0 acceptor_acceptor_quadratic(o=0,_c=8)	quadratic potential between hydrogen bond acceptors
1.0 donor_donor_quadratic(o=0,_c=8)	quadratic potential between hydroben bond donors
1.0  num_tors_div	div constant terms are not linearly independent
1.0  num_heavy_atoms_div
1.0  num_heavy_atoms	these terms are just added
1.0  num_tors_add
1.0  num_tors_sqr
1.0  num_tors_sqrt
1.0  num_hydrophobic_atoms
1.0  ligand_length
''');

In [ ]:
!./gnina -r rec.pdb -l lig.pdb --score_only --custom_scoring everything.txt

# CNN Scoring

In [ ]:
!./gnina --help | grep "cnn arg" -A 12

In [ ]:
!./gnina --score_only -r rec.pdb -l lig.pdb  | grep CNN

In [ ]:
%%time
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0  > /dev/null 2>&1

In [ ]:
%%time
!CUDA_VISIBLE_DEVICES= ./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb --seed 0

# Whole Protein Docking

In [ ]:
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand rec.pdb -o wdocking.sdf.gz --seed 0

In [ ]:
v = py3Dmol.view(height=400)
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(gzip.open('wdocking.sdf.gz','rt').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000}); v.zoomTo(); v.rotate(90)

# Flexible Docking

In [ ]:
!wget http://files.rcsb.org/download/4ERK.pdb

In [ ]:
!grep ATOM 4ERK.pdb > rec2.pdb
!obabel rec2.pdb -Orec2.pdb

In [ ]:
!grep OLO 4ERK.pdb > lig2.pdb

In [ ]:
!./gnina -r rec2.pdb -l lig.pdb --autobox_ligand lig2.pdb --seed 0 -o 3erk_to_4erk.sdf

In [ ]:
!obrms -firstonly lig.pdb 3erk_to_4erk.sdf

In [ ]:
!./gnina -r rec2.pdb -l lig.pdb --autobox_ligand lig2.pdb --seed 0 -o flexdocked.sdf --flexdist 4 --flexdist_ligand lig2.pdb --out_flex flexout.pdb

In [ ]:
!obrms -firstonly lig.pdb flexdocked.sdf

In [ ]:
!./gnina -r rec2.pdb -l lig.pdb --autobox_ligand lig2.pdb --seed 0 -o flexdocked2.sdf --exhaustiveness 16 --flexres A:52,A:103 --out_flex flexout2.pdb

# Virtual Screening

In [ ]:
!wget http://files.rcsb.org/download/4PPS.pdb

In [ ]:
!grep ^ATOM 4PPS.pdb > errec.pdb

In [ ]:
!wget http://bits.csb.pitt.edu/files/workshop_minimized_results.sdf.gz

In [ ]:
!./gnina -r errec.pdb -l workshop_minimized_results.sdf.gz --minimize -o gnina_scored.sdf.gz --scoring vinardo

In [ ]:
from openbabel import pybel
import pandas as pd

scores = []
for mol in pybel.readfile('sdf','gnina_scored.sdf.gz'):
    scores.append({'title': mol.title,
                   'CNNscore': float(mol.data['CNNscore']),
                   'CNNaffinity': float(mol.data['CNNaffinity']),
                   'Vinardo': float(mol.data['minimizedAffinity'])})
scores = pd.DataFrame(scores)
scores['label'] = scores.title.str.contains('active')

scores

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(dpi=150)
plt.plot([0,1],[0,1],'k--',alpha=0.5,linewidth=1)
fpr,tpr,_ = roc_curve(scores.label,-scores.Vinardo)
plt.plot(fpr,tpr,label="Vinardo (AUC = %.2f)"%auc(fpr,tpr))
fpr,tpr,_ = roc_curve(scores.label,scores.CNNaffinity)
plt.plot(fpr,tpr,label="CNNaffinity (AUC = %.2f)"%auc(fpr,tpr))
fpr,tpr,_ = roc_curve(scores.label, scores.CNNaffinity.rank() + (-scores.Vinardo).rank())
plt.plot(fpr,tpr,label="Consensus (AUC = %.2f)"%auc(fpr,tpr))
plt.legend(loc='lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.gca().set_aspect('equal')

# Custom Scoring

In [ ]:
!./gnina -r errec.pdb -l workshop_minimized_results.sdf.gz --score_only --custom_scoring everything.txt > scores.txt 2>&1

In [ ]:
!head -30 scores.txt

In [ ]:
import subprocess, io, re
terms = pd.read_csv(io.BytesIO(subprocess.check_output("grep \#\# scores.txt | sed 's/## //'",shell=True)),delim_whitespace=True)
terms[['CNNscore','CNNaffinity','CNNvariance']] = re.findall(r'CNNscore: (\S+)\s*CNNaffinity: (\S+)\s*CNNvariance: (\S+)',open('scores.txt').read())
terms['label'] = terms.Name.str.contains('active')

In [ ]:
import sklearn
from sklearn.linear_model import *

X = terms.drop(['Name','label'],axis=1).astype(float) # features
Y = terms.label

In [ ]:
X

In [ ]:
model = LogisticRegression(solver='liblinear')
cvpredict = sklearn.model_selection.cross_val_predict(model, X, Y, method='predict_proba')
fpr,tpr,_ = roc_curve(Y,cvpredict[:,1])
fig,ax = plt.subplots(1,1,dpi=100)
ax.plot(fpr,tpr,label="Combined CV ROC (AUC=%.2f)"%auc(fpr,tpr))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
ax.set_aspect('equal');